# CHOIX HYPERPARAMETRES Light SVC

LigthSVC est une modèle rapide et avec des tres bonnes performances.

On va essayer de trouver les meilleurs hyperparamettres en faisant un GridSearchCV

In [1]:
import pandas as pd

Importer les données:

In [2]:
df_X = pd.read_csv(r'C:\Users\Edgar\Documents\Rakuten\X_train\X_train_lemma-FR_stop_words-FR_no_num-FR_remove_accents-FR_no_special-FR_lemma-EN_stop_words-EN_stop_words-DE_lemma-DE_steem-FR_steem-EN_steem-DE.csv',index_col =0).squeeze()
df_y = pd.read_csv(r'C:\Users\Edgar\Documents\Rakuten\Y_train_CVw08PX.csv')

Transformer les categories target en numeros de 0 à 26:

In [3]:
y = df_y.prdtypecode
categories = list(set(y.to_list()))
y_trans = y
i = 0
for category in categories:
    y_trans = y_trans.replace(category,i)
    i+=1

y_trans

0         4
1        25
2        20
3         0
4         6
         ..
84911    17
84912    10
84913    25
84914    11
84915    23
Name: prdtypecode, Length: 84916, dtype: int64

Definition fonction qui calcule les nombre de mots uniques:

In [4]:
def number_unique_words(text_series): #count unique words in Series
    words = []
    for sentence in text_series.str.split():
        for word in sentence:
            words.append(word)
            
    return len(list(set(words)))

Separer les données en train et test:

In [5]:
# Importer la classe train_test 
from sklearn.model_selection import train_test_split

# Séparer le jeu de données en données d'entraînement et données test 
X_train, X_test, y_train, y_test = train_test_split(df_X, y_trans, test_size=0.2)

print('Validation size is',y_train.size ,'test size is', y_test.size)
print(number_unique_words(X_train))

Validation size is 67932 test size is 16984
78147


Transformer les données en vecteurs:

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

tfid = TfidfVectorizer(analyzer='word',
                       tokenizer=word_tokenize,
                       max_df=0.8,
                       min_df=4,
                       #ngram_range=(1,1),
                       use_idf=True,
                       smooth_idf=True,
                       sublinear_tf=False,
                       binary=True,
                      )

X_train_tf = tfid.fit_transform(X_train).todense()
X_test_tf = tfid.transform(X_test).todense()

X_train_tf.shape

(67932, 20936)

Creer la focntion pipeline qui va gérer la recherche d'hyperparamettres:

In [7]:
from sklearn.model_selection import GridSearchCV

#https://mlfromscratch.com/gridsearch-keras-sklearn/#/

def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=3, scoring_fit='f1_weighted',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
       # n_jobs=-1, 
        scoring=scoring_fit,
        verbose=3
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

LAncer le modèle choisi avec tous les possibles paramettres:

In [8]:
from sklearn.svm import LinearSVC

model = LinearSVC(penalty="l2", dual=True)

param_grid = {
    'dual':[False,True],
    'multi_class':['ovr', 'crammer_singer'],
    'C':[0.01,0.1,0.5],
    'max_iter':[50,100,500,1000],
    'class_weight':['balanced',None]
    }

model, pred = algorithm_pipeline(X_train_tf, X_test_tf, y_train, y_test, model, 
                                 param_grid, cv=3, scoring_fit='f1_weighted')

print(model.best_score_)
print(model.best_params_)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] C=0.01, class_weight=balanced, dual=False, max_iter=50, multi_class=ovr 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.01, class_weight=balanced, dual=False, max_iter=50, multi_class=ovr, score=0.752, total= 2.6min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV] C=0.01, class_weight=balanced, dual=False, max_iter=50, multi_class=ovr 
[CV]  C=0.01, class_weight=balanced, dual=False, max_iter=50, multi_class=ovr, score=0.751, total= 2.4min
[CV] C=0.01, class_weight=balanced, dual=False, max_iter=50, multi_class=ovr 


...

[CV]  C=0.5, class_weight=None, dual=True, max_iter=1000, multi_class=crammer_singer, score=0.820, total= 3.7min
[CV] C=0.5, class_weight=None, dual=True, max_iter=1000, multi_class=crammer_singer 


C:\Users\Edgar\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV]  C=0.5, class_weight=None, dual=True, max_iter=1000, multi_class=crammer_singer, score=0.818, total= 3.4min


[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed: 894.0min finished


0.8200530736890056
{'C': 0.5, 'class_weight': None, 'dual': False, 'max_iter': 50, 'multi_class': 'ovr'}


In [9]:
print('Best score:' ,model.best_score_, 'with hyperparameters:', model.best_params_)

Best score: 0.8200530736890056 with hyperparameters: {'C': 0.5, 'class_weight': None, 'dual': False, 'max_iter': 50, 'multi_class': 'ovr'}


On va essayer d'ajuster encore plus ces hyperparamettres autour du Best Score:

In [10]:
from sklearn.svm import LinearSVC

model = LinearSVC(penalty="l2", dual=True)

param_grid = {
    'dual':[False],
    'multi_class':['ovr'],
    'C':[0.3,0.4,0.5,0.6,0.7],
    'max_iter':[30,40,50,60,70],
    'class_weight':[None]
    }

model, pred = algorithm_pipeline(X_train_tf, X_test_tf, y_train, y_test, model, 
                                 param_grid, cv=3, scoring_fit='f1_weighted')

print('')
print('**********************************************')
print('Best score:' ,model.best_score_, 'with hyperparameters:', model.best_params_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.3, class_weight=None, dual=False, max_iter=30, multi_class=ovr 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.3, class_weight=None, dual=False, max_iter=30, multi_class=ovr, score=0.819, total= 3.2min
[CV] C=0.3, class_weight=None, dual=False, max_iter=30, multi_class=ovr 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s


[CV]  C=0.3, class_weight=None, dual=False, max_iter=30, multi_class=ovr, score=0.819, total= 2.7min
[CV] C=0.3, class_weight=None, dual=False, max_iter=30, multi_class=ovr 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.9min remaining:    0.0s


[CV]  C=0.3, class_weight=None, dual=False, max_iter=30, multi_class=ovr, score=0.817, total= 2.6min
[CV] C=0.3, class_weight=None, dual=False, max_iter=40, multi_class=ovr 
[CV]  C=0.3, class_weight=None, dual=False, max_iter=40, multi_class=ovr, score=0.819, total= 2.7min
[CV] C=0.3, class_weight=None, dual=False, max_iter=40, multi_class=ovr 
[CV]  C=0.3, class_weight=None, dual=False, max_iter=40, multi_class=ovr, score=0.819, total= 2.7min
[CV] C=0.3, class_weight=None, dual=False, max_iter=40, multi_class=ovr 
[CV]  C=0.3, class_weight=None, dual=False, max_iter=40, multi_class=ovr, score=0.817, total= 2.6min
[CV] C=0.3, class_weight=None, dual=False, max_iter=50, multi_class=ovr 
[CV]  C=0.3, class_weight=None, dual=False, max_iter=50, multi_class=ovr, score=0.819, total= 2.8min
[CV] C=0.3, class_weight=None, dual=False, max_iter=50, multi_class=ovr 
[CV]  C=0.3, class_weight=None, dual=False, max_iter=50, multi_class=ovr, score=0.819, total= 2.7min
[CV] C=0.3, class_weight=None

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 216.6min finished



**********************************************
Best score: 0.8207794100438032 with hyperparameters: {'C': 0.6, 'class_weight': None, 'dual': False, 'max_iter': 30, 'multi_class': 'ovr'}


# CONCLUSIONS

Il semble que al meilleure ponctuation qu'on peut avoir avec le LinearSVC est de 0.82